In [ ]:
import boto3
import re
from tqdm import tqdm
from collections import defaultdict

In [ ]:
CLIENT = boto3.client('athena', **credentials)
GLUE_CLIENT = boto3.client('glue', 'us-east-1')
DB = 'RD_EXT_A_RAIABD'
CATALOG = 'AwsDataCatalog'

In [ ]:
class Credentials:
    def __init__(self, region_name, profile_name, aws_access_key_id, aws_secret_access_key, aws_session_token):
        self.region_name = region_name
        self.profile_name = profile_name
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.aws_session_token = aws_session_token
    
    @classmethod
    def from_file(cls, path):
        config = configparser.ConfigParser()
        config.read(path)

        profile_name = config.sections()[0]
        aws_access_key_id = config.get(profile_name, 'aws_access_key_id')
        aws_secret_access_key = config.get(profile_name, 'aws_secret_access_key')
        aws_session_token = config.get(profile_name, 'aws_session_token')

        return cls('us-east-1', profile_name, aws_access_key_id, aws_secret_access_key, aws_session_token)
    
    def to_yaml(self, path):
        data = {
            'region_name': self.region_name,
            'profile_name': self.profile_name,
            'aws_access_key_id': self.aws_access_key_id,
            'aws_secret_access_key': self.aws_secret_access_key,
            'aws_session_token': self.aws_session_token
        }

        with open(path, 'w') as outfile:
            yaml.dump(data, outfile)
    
    @classmethod
    def from_yaml(cls, path):
        with open(path, 'r') as f:
            data = yaml.safe_load(f)

        return cls(data['region_name'], data['profile_name'], data['aws_access_key_id'], data['aws_secret_access_key'], data['aws_session_token'])

def get_databases(client = CLIENT,CatalogName = 'AwsDataCatalog'):
    response_lsdb = client.list_databases(
        CatalogName = CatalogName    )

def describe_table(result):
    tabela={}
    for i in result['ResultSet']['Rows']:
        values = i['Data'][0]['VarCharValue'].split()
        if len(values)>1: 
            tabela[values[0]] = values[1]
    return tabela

def get_table_cols(result):
    cols=[]
    for i in result['ResultSet']['Rows']:
        values = i['Data'][0]['VarCharValue'].split()
        if len(values)>1: 
            cols.append(values[0])
    return cols

def get_cv_cols(df):
    cols_raw = [re.findall('\[(.*?)\]', i) for i in df.columns.values]
    cols=[]
    for i in cols_raw:
        cols.append(i[0].lower())
    return cols
def get_databases(client = CLIENT,CatalogName = 'AwsDataCatalog'):
    response_lsdb = client.list_databases(
        CatalogName = CatalogName
    )
    databases = [i['Name'] for i in response_lsdb['DatabaseList']]
    return databases
def get_tables(client = CLIENT, CatalogName = 'AwsDataCatalog', DatabaseName = None):
    response_tables = client.list_table_metadata(
        CatalogName = CatalogName,
        DatabaseName = DatabaseName
    )
    tables = [i['Name'] for i in response_tables['TableMetadataList']]
    return tables
def get_at_columns(glue_client = GLUE_CLIENT, DatabaseName = None, Name = None):

    response_cols = glue_client.get_table(
        DatabaseName = DatabaseName,
        Name = Name
    )
    cols = [i['Name'] for i in response_cols['Table']['StorageDescriptor']['Columns']]
    return cols

def isin(word, string):
    if word.lower() in string.lower():
        return True
    else:
        return False

def search_col(db,list_cols):
    list_cols = ['cupom']
    db_found = defaultdict(dict)
    for db in tqdm(get_databases()):    
        for table in get_tables(DatabaseName = db):
            n_found_cols = 0
            found_cols = []
            for col_db in get_at_columns(DatabaseName=db, Name = table):           
                for wanted_col in list_cols:
                    if isin(wanted_col, col_db):
                        found_cols.append(col_db)
                        n_found_cols+=1
            if n_found_cols>0:
                db_found[db][table] = (n_found_cols, found_cols)
                        #print(db, table, n_found_cols)
    return db_found


def search_table(db,list_tables):
    list_tables= ['cupom']
    db_found = defaultdict(dict)
    for db in tqdm(get_databases()):    
        n_found_tables = 0
        found_tables = []
        for table in get_tables(DatabaseName = db):
            for wanted_table in list_tables:
                if isin(wanted_table, table):
                    found_tables.append(table)
                    n_found_tables+=1
        if n_found_tables>0:
            db_found[db] = (n_found_tables, found_tables)
    return db_found

In [ ]:
from pyprojroot import here
from elog.elog import logexc
from elog.elog import logtime
import os
LIB_PATH = os.path.join(here(),'src','aws')
print(LIB_PATH)
import sys
sys.path.append(LIB_PATH)
# from ed import search_col, search_table

@logexc
@logtime
def check_search_cols(db, list_cols):
    found_cols = search_col(db, list_cols)


@logexc
@logtime
def check_search_table(db, list_tables):
    found_cols = search_col(db, list_tables)

LIST_COLS = ['cupom']
LIST_TABLES = ['calendario']
check_search_cols(DB, LIST_COLS)
check_search_table(DB, LIST_TABLES)